In [3]:
from pathlib import Path
import re
import pandas as pd

def safe_sheet_name(name: str) -> str:
    """
    Excel sheet name rules:
    - Max 31 chars
    - Cannot contain: : \ / ? * [ ]
    - Cannot be empty
    """
    name = re.sub(r"[:\\/?*\[\]]", "_", name).strip()
    if not name:
        name = "Sheet"
    return name[:31]

def csvs_to_xlsx(csv_paths, output_xlsx: str):
    output_xlsx = str(output_xlsx)

    # Build unique sheet names (avoid duplicates after sanitizing/truncation)
    used = set()
    def unique_name(base: str) -> str:
        base = safe_sheet_name(base)
        candidate = base
        i = 2
        while candidate.lower() in used:
            suffix = f"_{i}"
            candidate = (base[:31 - len(suffix)] + suffix) if len(base) + len(suffix) > 31 else (base + suffix)
            i += 1
        used.add(candidate.lower())
        return candidate

    with pd.ExcelWriter(output_xlsx, engine="openpyxl") as writer:
        for p in csv_paths:
            p = Path(p)
            df = pd.read_csv(p)  # add encoding=... if needed
            sheet = unique_name(p.stem)
            df.to_excel(writer, sheet_name=sheet, index=False)

    print(f"Wrote {len(list(csv_paths))} CSV(s) into: {output_xlsx}")

In [7]:
# Option A: Merge all CSVs in a folder
folder = Path("")
csv_files = ["PS1.csv", "PS2.csv", "PS3.csv", "PS4.csv", "PS5.csv"]
csvs_to_xlsx(csv_files, "smal_questions_with_eval.xlsx")

Wrote 5 CSV(s) into: smal_questions_with_eval.xlsx
